# Make a reference database in FASTA format for an IPG Query

In [1]:
# Import packages
from Bio import Entrez
from Bio import SeqIO


# Set email adress
Entrez.email = "remi.legrand38@gmail.com.com"


## 1. Gather ID of IPG reports from a QUERY (2 sec)

In [2]:
# Function to search data on NCBI
def search_id(query, number_seq):
    handle = Entrez.esearch(db="ipg", term=query,
                            retmax=number_seq)  # number of sequences
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]


In [3]:
# Create a list containing the ID
query = "((nifH[Gene Name]) AND 100:350[Sequence Length]) NOT uncultured"
seq_id = search_id(query, 20000)
print(seq_id)
# len(seq_id)


['15644230', '15422654', '469614905', '463335738', '463307954', '463296292', '461244997', '461241703', '446901641', '427477591', '427469729', '427466342', '427423626', '415604000', '350848217', '350787255', '349442739', '349436496', '299156976', '226445887', '194947467', '180173506', '180172798', '180171254', '180170887', '100437320', '71845111', '69701369', '69700817', '69697418', '63695862', '56757924', '52011023', '44394908', '39093004', '36733349', '32474742', '22048464', '6622016', '6145268', '885911', '436307', '173295', '364405625', '353120723', '350866440', '350778347', '350773083', '349936002', '349903660', '349902885', '349841539', '349838987', '349837590', '349811669', '349694507', '347358025', '326797342', '304648145', '294032489', '279337086', '274656235', '224994493', '214571543', '214571338', '204648154', '202205411', '202148507', '202146568', '202099735', '123537395', '122931342', '110709219', '109359436', '92962718', '53228297', '42037040', '39031999', '19703386', '197

## 2. Download the IPG report (92 min)

In [4]:
def ipg_report(ipg_accession):
    # Use the efetch function to retrieve the IPG record in XML format
    handle = Entrez.efetch(db="ipg", id=ipg_accession,
                           rettype="ipg", retmode="text")
    ipg_record = handle.read()

    # Save the IPG record to a file
    filename = f"ipg_report/ipg_{ipg_accession}.txt"
    with open(filename, "wb") as file:
        file.write(ipg_record)

    print(f"IPG record saved to '{filename}'")


In [5]:
for id in seq_id:
    ipg_report(id)
    # print(id)
# ipg_report('127919708')


IPG record saved to 'ipg_report/ipg_15644230.txt'
IPG record saved to 'ipg_report/ipg_15422654.txt'
IPG record saved to 'ipg_report/ipg_469614905.txt'
IPG record saved to 'ipg_report/ipg_463335738.txt'
IPG record saved to 'ipg_report/ipg_463307954.txt'
IPG record saved to 'ipg_report/ipg_463296292.txt'
IPG record saved to 'ipg_report/ipg_461244997.txt'
IPG record saved to 'ipg_report/ipg_461241703.txt'
IPG record saved to 'ipg_report/ipg_446901641.txt'
IPG record saved to 'ipg_report/ipg_427477591.txt'
IPG record saved to 'ipg_report/ipg_427469729.txt'
IPG record saved to 'ipg_report/ipg_427466342.txt'
IPG record saved to 'ipg_report/ipg_427423626.txt'
IPG record saved to 'ipg_report/ipg_415604000.txt'
IPG record saved to 'ipg_report/ipg_350848217.txt'
IPG record saved to 'ipg_report/ipg_350787255.txt'
IPG record saved to 'ipg_report/ipg_349442739.txt'
IPG record saved to 'ipg_report/ipg_349436496.txt'
IPG record saved to 'ipg_report/ipg_299156976.txt'
IPG record saved to 'ipg_report/i

## 3. Extract the first line of the IPG report present in NCBI and put it in a table (23 sec)

- extract first line
- make a table with the accession number, star, stop and the strand of the sequence 

In [6]:
import pandas as pd
import numpy as np


gene_list = pd.DataFrame(columns=["accession", "start", "stop", "strand"])

for name in seq_id:
    file = "ipg_report/ipg_"+name+".txt"
    ipg = pd.read_csv(file, sep="\t")

    insdc = np.array(ipg.loc[:, "Source"] == "INSDC", dtype="bool")
    refseq = np.array(ipg.loc[:, "Source"] == "RefSeq", dtype="bool")

    in_ncbi = ipg.loc[refseq | insdc, :]
    in_ncbi = in_ncbi.reset_index()
    url = str()
    if len(in_ncbi) != 0:
        name = str(in_ncbi.loc[0, "Nucleotide Accession"])
        start = str(in_ncbi.loc[0, "Start"])
        stop = str(in_ncbi.loc[0, "Stop"])
        if str(in_ncbi.loc[0, "Strand"]) == "+":
            strand = "1"
        elif str(in_ncbi.loc[0, "Strand"]) == "-":
            strand = "2"
        gene_list.loc[len(gene_list)] = [name, start, stop, strand]
    else:
        print(file+" is not in NCBI")

gene_list


,accession,start,stop,strand
0,NC_011283.1,1769523.0,1770404.0,2
1,NC_017174.1,1894823,1895572,1
2,NZ_JAEEGC010000032.1,27209,27979,1
3,NZ_JAHLQL010000001.1,1591380,1592219,2
4,NZ_JAHLQK010000001.1,563737,564495,1
...,...,...,...,...
3922,DQ284983.1,1,800,1
3923,AY603652.1,1,580,1
3924,AJ302315.1,1,863,1
3925,U66032.1,4945,5643,1


##  3 V2

In [7]:
# import pandas as pd
# import numpy as np


# gene_list = pd.DataFrame(columns=["accession", "start", "stop", "strand"])

# for name in seq_id:
#     file = "ipg_report/ipg_"+name+".txt"
#     ipg = pd.read_csv(file, sep="\t")

#     insdc = np.array(ipg.loc[:, "Source"] == "INSDC", dtype="bool")
#     refseq = np.array(ipg.loc[:, "Source"] == "RefSeq", dtype="bool")

#     in_ncbi = ipg.loc[refseq | insdc, :]
#     in_ncbi = in_ncbi.reset_index()
#     url = str()
#     if len(in_ncbi) != 0:
#         name = str(in_ncbi.loc[0, "Nucleotide Accession"])
#         start = str(in_ncbi.loc[0, "Start"])
#         stop = str(in_ncbi.loc[0, "Stop"])
#         if str(in_ncbi.loc[0, "Strand"]) == "+":
#             strand = "1"
#         elif str(in_ncbi.loc[0, "Strand"]) == "-":
#             strand = "2"
#         gene_list.loc[len(gene_list)] = [name, start, stop, strand]
#     else:
#         print(file+" is not in NCBI")

# gene_list

## 4. Fetch with ENTREZ the INSDSeq XML report for each line of the previous table and download it (1h51min)

In [8]:
from Bio import Entrez

Entrez.email = "remi.legrand38@gmail.com.com"


def download_insdseq_xml(index):
    accession = gene_list.loc[index, "accession"]
    start = gene_list.loc[index, "start"]
    stop = gene_list.loc[index, "stop"]
    strand = gene_list.loc[index, "strand"]
    handle = Entrez.efetch(db='nucleotide', id=accession, rettype='gb',
                           retmode='xml', seq_start=start, seq_stop=stop, strand=strand)
    # Save the XML data to a file
    with open(f'insdseq_xml_report/{accession}_INSDSeq.xml', 'wb') as file:
        file.write(handle.read())

    print('INSDSeq XML report downloaded successfully.')


for i in range(len(gene_list)):
    download_insdseq_xml(i)


INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded successfully.
INSDSeq XML report downloaded su

## 5. Make a FASTA file by extracting for each file the taxa and the seq (25 sec)
some files like NC_007512.1_INSDSeq.xml have a missing either sequence or taxomnomy: fixed with Try but i this case, it would be nice to search for an other sequence in the sequence list

In [10]:
import os
import xml.etree.ElementTree as ET

files = os.listdir('insdseq_xml_report')

with open('reference_db_ipg.fasta', 'w') as file:
    for xml_file in files:
        path = 'insdseq_xml_report/' + xml_file
        # print(path)
        tree = ET.parse(path)
        root = tree.getroot()
        try:
            tax = root.find("./GBSeq/GBSeq_taxonomy").text
            seq = root.find("./GBSeq/GBSeq_sequence").text
        except AttributeError:
            print("in", path, "sequence or taxonomy not found")
        # print(tax)
        # print(seq)
        file.write(">" + tax + "\n")
        file.write(seq + "\n")


in insdseq_xml_report/NC_007512.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NZ_JACDUL010000003.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NZ_ATNG01000008.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_014658.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_014002.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_002678.2_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_003901.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_005791.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_003030.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_007778.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_009434.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NC_011565.1_INSDSeq.xml sequence or taxonomy not found
in insdseq_xml_report/NZ_CM001436.1_INSDSeq.xml sequence or t

files with problems on the first half of the document(sequence too long):

insdseq_xml_report/NZ_JAGGMU010000001.1_INSDSeq.xml
insdseq_xml_report/NC_008576.1_INSDSeq.xml
insdseq_xml_report/NC_007681.1_INSDSeq.xml
insdseq_xml_report/NC_018227.2_INSDSeq.xml
insdseq_xml_report/J05111.1_INSDSeq.xml **contain nifU, nifS, nifB, nifH, etc**
insdseq_xml_report/NC_016617.1_INSDSeq.xml
insdseq_xml_report/NC_009523.1_INSDSeq.xml
insdseq_xml_report/NC_014323.1_INSDSeq.xml
insdseq_xml_report/CP003083.1_INSDSeq.xml
insdseq_xml_report/NZ_JMIY01000005.1_INSDSeq.xml
insdseq_xml_report/NC_011206.1_INSDSeq.xml
insdseq_xml_report/NC_004041.2_INSDSeq.xml
insdseq_xml_report/CP003284.1_INSDSeq.xml
insdseq_xml_report/NC_009253.1_INSDSeq.xml
insdseq_xml_report/NC_000916.1_INSDSeq.xml
insdseq_xml_report/NC_011283.1_INSDSeq.xml

## 6. Normalize the taxonomy (If I have the time): remove the sub classes